# Generative Model Project
Yassine Ben Jemaa
Irshath Nagouri

# Dataset 


In [111]:
import pandas as pd

# Load the CSV file into a DataFrame
file_path = './data_train_log_return.csv'
data_train_log_return = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
data_train_log_return.head()


,0,0.01249535315117,0.0111256706670408,0.0032520459252687,0.0066249108779032
0,1,0.011439,0.002691,0.001206,0.006947
1,2,0.000632,0.007277,0.004049,0.000074
2,3,0.017828,0.028210,0.007758,0.007382
3,4,0.021115,0.019642,0.009238,0.011499
4,5,0.001177,0.002096,0.001348,0.004966


In [112]:
df = data_train_log_return
df = df.drop(columns = ["0"])
df = df.rename(columns={'0.01249535315117': 'f1', '0.0111256706670408': 'f2', '0.0032520459252687': 'f3', '0.0066249108779032': 'f4'})
df



,f1,f2,f3,f4
0,0.011439,0.002691,0.001206,0.006947
1,0.000632,0.007277,0.004049,0.000074
2,0.017828,0.028210,0.007758,0.007382
3,0.021115,0.019642,0.009238,0.011499
4,0.001177,0.002096,0.001348,0.004966
...,...,...,...,...
740,0.001938,0.008833,0.003927,0.005106
741,0.005003,0.018943,0.003057,0.001988
742,0.007683,0.001958,0.007002,0.006467
743,0.003396,0.001280,0.007621,0.001680


# VAE Implementation 


In [113]:
from keras.models import load_model
vae2 = load_model('vae_model.h5', compile = False)
vae2.compile(optimizer='adam')
file_path = './noise.csv'
noise = pd.read_csv(file_path)
noise = noise.to_numpy()
vae2.summary()

Model: "vae"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_35 (InputLayer)       [(None, 4)]                  0         []                            
                                                                                                  
 encoder (Functional)        [(None, 3),                  45242     ['input_35[0][0]']            
                              (None, 3),                                                          
                              (None, 3)]                                                          
                                                                                                  
 decoder (Functional)        (None, 4)                    11588     ['encoder[0][2]']             
                                                                                                

In [114]:
decoder = vae2.get_layer('decoder')
z_sample = noise # Generate latent space points from the csv file
generated_data = decoder.predict(z_sample) 

13/13 [==============================] - 0s 1ms/step


In [115]:
data_410 = df.head(410)
ori_data = data_410.to_numpy()
print(type(data_410))

<class 'pandas.core.frame.DataFrame'>


# Metrics Evaluation 

In [116]:
import numpy as np
# Anderson-Darling Distance
def anderson_darling_distance(generated_array, original_array):

    def indice(a, b):
        indice = int(a <= b)  
        return indice

    def u_i(X,Y):
        n=len(X)
        X.sort()
        l=[]
        for i in range(n):
            l.append((1/(n+2))*(sum(indice(y,X[i]) for y in Y)+1))
        return l
    
    def W(X,Y):
        # Wn is a statistic that measures the distance between the empirical distribution function of the sample and the theoretical distribution function
        n=len(X)
        u=u_i(X,Y)
        w_n=-n-(1/n)*sum(((2*i-1)*(np.log(u[i-1])+np.log(1-u[n-i]))) for i in range(1,n+1)) 
        return w_n

    def anderson_darling(x,y):
        # For the final Anderson-Darling distance, we average the Wn values for each feature
        d=x.shape[1]
        W_n=(1/d)*sum(W(x[i],y[i]) for i in range(d))
        return W_n


    return anderson_darling(generated_array, original_array)

anderson_darling_distance(generated_data, ori_data)

1.1113128618497

In [117]:
#Absolute Kendall error
import numpy as np

# Function to calculate Z_i for a dataset
def calculate_zi(data):
    n, d = data.shape
    zi = np.zeros(n)
    for i in range(n):
        for j in range(n):
            if j != i:
                zi[i] += np.sum(data[j, :] < data[i, :])
        zi[i] /= (n - 1) * d
    return zi

# Function to calculate the Absolute Kendall error between two datasets
def absolute_kendall_error(data1, data2):
    # Ensure the datasets have the same size
    if data1.shape != data2.shape:
        raise ValueError("Datasets must have the same size")
    
    # Calculate Z_i for both datasets
    zi_data1 = calculate_zi(data1)
    zi_data2 = calculate_zi(data2)
    
    # Compute the L1 norm of the difference between the Z_i vectors
    absolute_error = np.mean(np.abs(zi_data1 - zi_data2))
    return absolute_error



# Calculate the Absolute Kendall error
error = absolute_kendall_error(ori_data, generated_data)
print(f"Absolute Kendall error: {error}")


Absolute Kendall error: 0.22743753354403956
